<a href="https://colab.research.google.com/github/KhresnaPanduI/ASL-Model/blob/main/asl_model_ver1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup

In [5]:
import numpy as np
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub
from keras.preprocessing.image import ImageDataGenerator

from tqdm import tqdm


In [6]:
module_selection = ("mobilenet_v2", 224, 1280) #@param ["(\"mobilenet_v2\", 224, 1280)", "(\"inception_v3\", 299, 2048)"] {type:"raw", allow-input: true}
handle_base, pixels, FV_SIZE = module_selection
MODULE_HANDLE ="https://tfhub.dev/google/tf2-preview/{}/feature_vector/4".format(handle_base)
IMAGE_SIZE = (pixels, pixels)
print("Using {} with input size {} and output dimension {}".format(MODULE_HANDLE, IMAGE_SIZE, FV_SIZE))

Using https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4 with input size (224, 224) and output dimension 1280


Data extraction

In [7]:
train_dir = '/content/drive/MyDrive/Datasets/own asl/Train'
test_dir = '/content/drive/MyDrive/Datasets/own asl/test'

Number of class

In [8]:
import os

categories = os.listdir(train_dir)
categoriest = os.listdir(test_dir)
print(len(categories))
print(len(categoriest))

29
29


In [9]:
train_data_gen = ImageDataGenerator(
                              rescale=1/255)

test_data_gen = ImageDataGenerator(
                              rescale=1/255)

In [10]:
train_gen = train_data_gen.flow_from_directory(train_dir,
                                         target_size=(224,224),
                                         class_mode="categorical",
                                         color_mode="rgb",
                                         shuffle=True,
                                         batch_size=32)

val_gen = test_data_gen.flow_from_directory(test_dir,
                                          target_size=(224,224),
                                          class_mode="categorical",
                                          color_mode="rgb",
                                          shuffle=True,
                                          batch_size=32,)

Found 1171 images belonging to 29 classes.
Found 289 images belonging to 29 classes.


In [11]:
feature_extractor = hub.KerasLayer(MODULE_HANDLE,
                                   input_shape=IMAGE_SIZE + (3,), 
                                   output_shape=[FV_SIZE],
                                   trainable=True)

In [12]:
print("Building model with", MODULE_HANDLE)

model = tf.keras.Sequential([
        feature_extractor,
        tf.keras.layers.Dense(29, activation='softmax')
])

model.summary()

Building model with https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 29)                37149     
Total params: 2,295,133
Trainable params: 2,261,021
Non-trainable params: 34,112
_________________________________________________________________


Unfreeze some layer

In [13]:
NUM_LAYERS = 10 #@param {type:"slider", min:1, max:50, step:1}
      
feature_extractor.trainable = True
    
for layer in model.layers[-NUM_LAYERS:]:
  layer.trainable = True

Training the model

In [17]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics=['accuracy'])

hist = model.fit(train_gen,
                 epochs = 10,
                 validation_data = val_gen)

Epoch 1/10
37/37 [==============================] - 23s 426ms/step - loss: 0.4161 - accuracy: 0.9175 - val_loss: 1.5601 - val_accuracy: 0.6920
Epoch 2/10
37/37 [==============================] - 15s 402ms/step - loss: 0.1962 - accuracy: 0.9817 - val_loss: 12.8974 - val_accuracy: 0.0969
Epoch 3/10
37/37 [==============================] - 15s 402ms/step - loss: 0.0984 - accuracy: 0.9994 - val_loss: 1.2362 - val_accuracy: 0.7543
Epoch 4/10
37/37 [==============================] - 15s 404ms/step - loss: 0.1184 - accuracy: 0.9949 - val_loss: 0.8173 - val_accuracy: 0.7958
Epoch 5/10
37/37 [==============================] - 15s 404ms/step - loss: 0.1132 - accuracy: 0.9961 - val_loss: 4.7583 - val_accuracy: 0.4706
Epoch 6/10
37/37 [==============================] - 15s 404ms/step - loss: 0.1234 - accuracy: 0.9945 - val_loss: 0.4853 - val_accuracy: 0.9031
Epoch 7/10
37/37 [==============================] - 15s 405ms/step - loss: 0.0995 - accuracy: 0.9979 - val_loss: 1.6809 - val_accuracy: 0.654

In [21]:
ASL_SAVED_MODEL = "asl_saved_model"

In [22]:
tf.saved_model.save(model, ASL_SAVED_MODEL)

INFO:tensorflow:Assets written to: asl_saved_model/assets


INFO:tensorflow:Assets written to: asl_saved_model/assets


In [24]:
%%bash -s $ASL_SAVED_MODEL
saved_model_cli show --dir $1 --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['keras_layer_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 224, 224, 3)
      name: serving_default_keras_layer_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 29)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [26]:
loaded = tf.saved_model.load(ASL_SAVED_MODEL)

print(list(loaded.signatures.keys()))
infer = loaded.signatures["serving_default"]
print(infer.structured_input_signature)
print(infer.structured_outputs)

['serving_default']
((), {'keras_layer_input': TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_layer_input')})
{'dense': TensorSpec(shape=(None, 29), dtype=tf.float32, name='dense')}


In [28]:
converter = tf.lite.TFLiteConverter.from_saved_model(ASL_SAVED_MODEL)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()

In [34]:
tflite_model_file = 'converted_asl_model.tflite'

with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

Create file to save labels

In [35]:
class_names = ['a', 'b', 'c', 'd', 'e', 'del', 'f', 'g', 'h',
               'i', 'j', 'k', 'l', 'm', 'n', 'nothing', 'o', 'p',
               'q', 'r', 's', 'space', 't', 'u', 'v', 'w', 'x', 
               'y', 'z']

with open('labels.txt', 'w') as f:
    f.write('\n'.join(class_names))

In [36]:
try:
    from google.colab import files
    files.download('converted_asl_model.tflite')
    files.download('labels.txt')
except:
    pass



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>